In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
target = pd.read_csv('target.csv')
display(numerical.head(5))
display(categorical.head(5))
display(target.head(5))

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3


,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [3]:
donors = pd.concat([numerical, categorical, target], axis=1)
donors.head(5)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,...,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,...,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,...,91,11,92,7,95,12,90,1,0,0.0
3,0,70.000000,1,4,2,0,23,14,31,3,...,87,11,94,11,95,12,87,2,0,0.0
4,0,78.000000,3,2,60,1,28,9,53,26,...,93,10,96,1,96,1,79,3,0,0.0


In [4]:
# Confirming that is there any nulls.
donors.isna().sum().sum()

0

   + Apply the Random Forests algorithm but this time only by upscaling the data to deal with the imbalance.

In [5]:
X = donors.drop(columns=['TARGET_B','TARGET_D'])
y = donors['TARGET_B']

In [6]:
print(X.shape)
print(y.shape)

(95412, 337)
(95412,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
#Imbalance check
print("Imbalance Check:")
print(y_train.value_counts())

Imbalance Check:
0    72419
1     3910
Name: TARGET_B, dtype: int64


In [9]:
train_data = pd.concat([X_train, y_train], axis=1)
#trainset = pd.concat([X_train, y_train], axis=1)

In [10]:
from sklearn.utils import resample
class_0 = train_data[train_data['TARGET_B'] == 0]
class_1 = train_data[train_data['TARGET_B'] == 1]

#category_0 = trainset[trainset['TARGET_B']==0].sample(len(trainset[trainset['TARGET_B']==1]))
#category_1 = trainset[trainset['TARGET_B']== 1 ]

In [11]:
class_1_upsampled = resample(class_1, 
                                   replace=True,    
                                   n_samples=len(class_0))

In [12]:
upsampled_data = pd.concat([class_0, class_1_upsampled])
upsampled_data = upsampled_data.sample(frac =1)

In [13]:
upsampled_data['TARGET_B'].value_counts()

1    72419
0    72419
Name: TARGET_B, dtype: int64

In [14]:
upsampled_data.shape

(144838, 338)

In [15]:
X_train_upsampled = upsampled_data.drop('TARGET_B', axis=1)
y_train_upsampled = upsampled_data['TARGET_B']

#X_train = trainset_new.drop(['TARGET_B'], axis=1)
#y_train = trainset_new['TARGET_B']

In [16]:
# Check the number of rows
print(X_train_upsampled.shape)
print(X_test.shape)
print(y_train_upsampled.shape)
print(y_test.shape)

(144838, 337)
(19083, 337)
(144838,)
(19083,)


In [17]:
X_train_num = X_train_upsampled.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)
X_train_cat = X_train_upsampled.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [18]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) 
X_train_scaled_arr = transformer.transform(X_train_num)
X_train_scaled = pd.DataFrame(X_train_scaled_arr, columns=X_train_num.columns)
#X_train_scaled.head()
X_train_scaled.shape

(144838, 330)

In [19]:
transformer = MinMaxScaler().fit(X_test_num) 
X_test_scaled_arr = transformer.transform(X_test_num)
X_test_scaled = pd.DataFrame(X_test_scaled_arr, columns=X_test_num.columns)
#X_test_scaled.head()
X_test_scaled.shape

(19083, 330)

In [20]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first')

# Fit and Transform X_train
encoded_train_cat = encoder.fit_transform(X_train_cat).toarray()
cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
onehot_encoded_train = pd.DataFrame(encoded_train_cat, columns=cols)

# Transform X_test
encoder.set_params(handle_unknown='ignore')
encoded_test_cat = encoder.transform(X_test_cat).toarray()
onehot_encoded_test = pd.DataFrame(encoded_test_cat, columns=cols)

In [21]:
# Dataframe from X_train
X_train_treated = pd.concat([X_train_scaled, onehot_encoded_train], axis=1)
X_train_treated

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.000014,0.804124,0.666667,0.888889,0.008299,0.0,0.252525,0.363636,0.363636,0.050505,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.013916,0.624862,0.500000,0.111111,0.000000,0.0,0.393939,0.262626,0.494949,0.090909,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000389,0.624862,0.666667,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.000000,0.556701,0.666667,1.000000,0.000000,0.0,0.282828,0.545455,0.202020,0.080808,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000028,0.628866,1.000000,0.888889,0.029046,0.0,0.090909,0.484848,0.000000,0.050505,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144833,0.000014,0.515464,1.000000,1.000000,0.000000,0.0,0.424242,0.222222,0.383838,0.121212,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
144834,0.000028,0.624862,0.166667,0.444444,0.016598,0.0,0.262626,0.343434,0.131313,0.060606,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
144835,0.000000,0.670103,0.000000,0.333333,0.004149,0.0,0.292929,0.171717,0.404040,0.040404,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
144836,0.000028,0.443299,0.500000,1.000000,0.000000,0.0,0.424242,0.050505,0.686869,0.040404,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
# Dataframe from X_test
X_test_treated = pd.concat([X_test_scaled, onehot_encoded_test], axis=1)
X_test_treated

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.000000,0.624862,0.666667,1.000000,0.000000,0.000000,0.404040,0.272727,0.303030,0.118644,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.000026,0.624862,0.166667,1.000000,0.000000,0.000000,0.262626,0.313131,0.181818,0.101695,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000718,0.865979,0.833333,1.000000,0.000000,0.000000,0.111111,0.202020,0.383838,0.016949,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.000051,0.624862,0.500000,0.666667,0.012448,0.000000,0.292929,0.656566,0.101010,0.135593,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.000718,0.624862,0.666667,1.000000,0.000000,0.000000,0.585859,0.212121,0.484848,0.152542,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,0.000026,0.690722,0.833333,1.000000,0.000000,0.000000,0.262626,0.242424,0.343434,0.084746,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19079,0.000000,0.624862,0.000000,0.333333,0.016598,0.000000,0.363636,0.242424,0.383838,0.220339,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19080,0.000000,0.690722,0.166667,0.555556,0.120332,0.000000,0.333333,0.101010,0.515152,0.067797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19081,0.000000,0.855670,0.333333,0.333333,0.078838,0.222222,0.444444,0.282828,0.222222,0.118644,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [23]:
#X_train_upsampled.head()
#X_train_upsampled.dtypes

In [24]:
y_train_upsampled.head()

81685    1
15885    0
80128    1
85862    0
23709    0
Name: TARGET_B, dtype: int64

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(X_train_treated, y_train_upsampled)
print(clf.score(X_train_treated, y_train_upsampled))
print(clf.score(X_test_treated, y_test))


0.6228752123061627
0.6904574752397422


In [26]:
y_pred = clf.predict(X_test_treated)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0    18150
1      933
Name: TARGET_B, dtype: int64

array([[12762,  5388],
       [  519,   414]], dtype=int64)

   + Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)

In [27]:
X_train_treated.reset_index(drop=True, inplace=True)
y_train_upsampled.reset_index(drop=True, inplace=True)
X_test_treated.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [38]:
data_train = pd.concat([X_train_treated, y_train_upsampled], axis=1)
data_train.head(5)
#print(data_train.isna().sum().sum())

0


In [40]:
data_test = pd.concat([X_test_treated, y_test], axis=1)
data_test.head(5)
#print(data_test.isna().sum().sum())

0


In [43]:
data_fea = pd.concat([data_train,data_test], axis=0)
data_fea

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,0.000014,0.804124,0.666667,0.888889,0.008299,0.000000,0.252525,0.363636,0.363636,0.050505,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,0.013916,0.624862,0.500000,0.111111,0.000000,0.000000,0.393939,0.262626,0.494949,0.090909,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,0.000389,0.624862,0.666667,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
3,0.000000,0.556701,0.666667,1.000000,0.000000,0.000000,0.282828,0.545455,0.202020,0.080808,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.000028,0.628866,1.000000,0.888889,0.029046,0.000000,0.090909,0.484848,0.000000,0.050505,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,0.000026,0.690722,0.833333,1.000000,0.000000,0.000000,0.262626,0.242424,0.343434,0.084746,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
19079,0.000000,0.624862,0.000000,0.333333,0.016598,0.000000,0.363636,0.242424,0.383838,0.220339,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19080,0.000000,0.690722,0.166667,0.555556,0.120332,0.000000,0.333333,0.101010,0.515152,0.067797,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
19081,0.000000,0.855670,0.333333,0.333333,0.078838,0.222222,0.444444,0.282828,0.222222,0.118644,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0


In [44]:
data_fea.isna().sum().sum()

0

In [45]:
X_data_fea = data_fea.drop('TARGET_B', axis=1)
y_data_fea = data_fea['TARGET_B']

   + Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.

   * Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?